In [7]:
import pandas as pd

# Read the .txt file line by line
with open(r'C:\Users\karan\Desktop\Amazon classification Project end to end\smart-reviews-classifier\data\test.ft.txt.csv', 'r', encoding='latin1') as f:
    lines = f.readlines()

# Parse the lines into labels and texts
data = []

for line in lines:
    if ' ' not in line:
        continue  # Skip bad/malformed lines
    
    label = 1 if '__label__2' in line else 0
    try:
        text = line.split(' ', 1)[1].strip()
        data.append((label, text))
    except IndexError:
        continue  # Just skip problematic lines


# Create DataFrame
df = pd.DataFrame(data, columns=['label', 'text'])
df.head()


,label,text
0,0,"ÿÿÿ¿ÿÿþÿÿÿÿ` (& ""(   e«ûèl KE   @..."
1,0,    H 
2,0,"¨DI""½´%R½¶Z«¬E^öê  ( I@ªJXéÐ@  ª-..."
3,0,Gfªª T@·ÛU¹öê¥%Dª* Rf¦° ­ØàR¨ËJ(
4,0,!R*\t*IQ@ $$³ ¥Ö


In [ ]:
df.shape